In [152]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
from numpy import zeros_like
from IPython.display import display
def display_matrix(m):
    display(sympy.Matrix(m))
import sympy
import pandas as pd
import scipy.stats as ss
from scipy.stats.qmc import Sobol
from scipy.stats import norm

sympy.init_printing()

In [153]:
basis = 'leguerre'

# parameters

mu = 0.06
r = 0.06
T = 1
I = 250
paths = 10000
S0 = 40
S0_list = range(37, 43, 1)
m = 0
K = 40

GBM_sigma = 0.04
GBM_mu = r

LN_lam = 1
LN_sigma = 0.02
LN_mu = r
LN_v = 0.02

JR_lam = 0.01
JR_sigma = 0.03
JR_mu = r + JR_lam

LNparams = (LN_lam, LN_sigma, LN_mu, LN_v, m)
JRparams = (JR_lam, JR_sigma, JR_mu)
GBMparams = (GBM_mu, GBM_sigma)

In [154]:
def generate_quasi_random_normal(number_of_samples, d):
    # Create a Sobol sequence generator
    sobol = Sobol(d, scramble=True)  # 1-dimensional sequence

    # Generate quasi-random numbers in the range [0, 1]
    quasi_random_uniform = sobol.random_base2(m=int(np.log2(number_of_samples)))

    # Transform the quasi-random numbers to a normal distribution
    return norm.ppf(quasi_random_uniform)

In [156]:
def merton_jump_paths_sobol(S0, paths, lam, sigma, mu, v, m, T, I, quasi_random_numbers):
    np.random.seed(42)
    matrix = np.zeros((paths, I))
    quasi_random_index = 0  # Index to track the current quasi-random number
    for k in range(paths):
        X = np.zeros(I)
        S = np.zeros(I)
        
        S[0] = S0
        X[0] = np.log(S0)
        
        dt = T / I
    
        for i in range(1,I):
            Z = quasi_random_numbers[quasi_random_index % len(quasi_random_numbers)]
            quasi_random_index += 1
            N = np.random.poisson(lam * dt)
            Y = np.exp(np.random.normal(m,v,N))
            #Y = np.random.lognormal(m,np.sqrt(v),N)
    
            if N == 0:
                M = 0
            else:
                M = np.sum(np.log(Y))
                
            X[i] = X[i-1] + (mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z + M
            S[i] = np.exp(X[i])
        matrix[k] = S
    return matrix

In [157]:
def merton_jump_paths(S0, paths, lam, sigma, mu, v, m, T, I):
    np.random.seed(42)
    matrix = np.zeros((paths, I))
    for k in range(paths):
        X = np.zeros(I)
        S = np.zeros(I)
        
        S[0] = S0
        X[0] = np.log(S0)
        
        dt = T / I
    
        for i in range(1,I):
            Z = np.random.standard_normal()
            N = np.random.poisson(lam * dt)
            Y = np.exp(np.random.normal(m,v,N))
            #Y = np.random.lognormal(m,np.sqrt(v),N)
    
            if N == 0:
                M = 0
            else:
                M = np.sum(np.log(Y))
                
            X[i] = X[i-1] + (mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z + M
            S[i] = np.exp(X[i])
        matrix[k] = S
    return matrix

In [158]:
# Total number of quasi-random samples needed
number_of_samples = paths * I

d = 1

# Generate the quasi-random numbers
quasi_random_numbers = generate_quasi_random_normal(number_of_samples, d)

In [159]:
def LSM(K, S, I, df, basis, deg):
    paths = len(S)
    np.random.seed(42)
    H = np.maximum(K - S, 0)  # intrinsic values for put option
    V = np.zeros_like(H)  # value matrix
    V[:, -1] = H[:, -1]  # set value at maturity equal to intrinsic value

    # Valuation by LS Method
    for t in range(I - 2, 0, -1):
        good_paths = H[:, t] > 0  # paths where the intrinsic value is positive

        if np.sum(good_paths) > 0:
            if basis == 'poly':
                rg = np.polyfit(S[good_paths, t], V[good_paths, t + 1] * df, deg)
                C = np.polyval(rg, S[good_paths, t])
            elif basis == 'legendre':
                rg = np.polynomial.legendre.legfit(S[good_paths, t], V[good_paths, t + 1] * df, deg)
                C = np.polynomial.legendre.legval(S[good_paths, t], rg)
            elif basis =='laguerre':
                rg = np.polynomial.laguerre.lagfit(S[good_paths, t], V[good_paths, t + 1] * df, deg)
                C = np.polynomial.laguerre.lagval(S[good_paths, t], rg)
            else:  # 'hermite'
                rg = np.polynomial.hermite.hermfit(S[good_paths, t], V[good_paths, t + 1] * df, deg)
                C = np.polynomial.hermite.hermval(S[good_paths, t], rg)

            exercise = np.zeros(len(good_paths), dtype=bool)
            exercise[good_paths] = H[good_paths, t] > C
        else:
            # If all intrinsic values are zero, mark all as non-exercise
            exercise = np.zeros(len(good_paths), dtype=bool)

        V[exercise, t] = H[exercise, t]
        V[exercise, t + 1 :] = 0
        discount_path = ~exercise
        V[discount_path, t] = V[discount_path, t + 1] * df

    V0 = np.mean(V[:, 1]) * df  # discounted expectation of V[t=1]
    V0_array = V[:, 1] * df
    SE = np.std(V[:, 1] * df) / np.sqrt(paths)
    variance = np.var(V[:, 1] * df)
    return V0, V0_array, SE, variance

In [160]:
quasi_ln_paths = merton_jump_paths_sobol(S0, paths, LN_lam, LN_sigma, LN_mu, LN_v, m, T, I, quasi_random_numbers)
ln_paths = merton_jump_paths(S0, paths, LN_lam, LN_sigma, LN_mu, LN_v, m, T, I)

In [161]:
V0_sobol, V0_array_sobol, SE_sobol, variance_sobol = LSM(K, quasi_ln_paths, I, np.exp(-r * T), basis, 3)
V0, V0_array, SE, variance = LSM(K, ln_paths, I, np.exp(-r * T), basis, 3)

print('V0: ', V0)
print('SE: ', SE)
print('Variance: ', variance)

print('V0_sobol: ', V0_sobol)
print('SE_sobol: ', SE_sobol)
print('Variance_sobol: ', variance_sobol)

V0:  0.038897606853536534
SE:  0.0008073871326428153
Variance:  0.006518739819571871
V0_sobol:  0.03233764339255106
SE_sobol:  0.0007658361366169534
Variance_sobol:  0.005865049881483811


In [162]:
#def generate_quasi_random_normal(paths, I, dimensions):
#    # Total number of quasi-random numbers needed
#    number_of_samples = paths * I * dimensions
#
#    # Create a Sobol sequence generator
#    sobol = Sobol(d=dimensions, scramble=True)
#
#    # Generate quasi-random numbers in the range [0, 1]
#    quasi_random_uniform = sobol.random_base2(m=int(np.ceil(np.log2(number_of_samples / dimensions))))
#    quasi_random_normal = norm.ppf(quasi_random_uniform)
#    return quasi_random_normal.reshape(-1, dimensions)
#
#def merton_jump_paths_sobol(S0, paths, lam, sigma, mu, v, m, T, I, max_jumps_per_step, quasi_random_numbers):
#    matrix = np.zeros((paths, I))
#    for k in range(paths):
#        X = np.zeros(I)
#        S = np.zeros(I)
#        S[0] = S0
#        X[0] = np.log(S0)
#        dt = T / I
#
#        for i in range(1, I):
#            idx = k * I + i - 1
#            Z = quasi_random_numbers[idx, 0]  # First dimension for GBM
#
#            N = np.random.poisson(lam * dt)
#            M = 0
#            for j in range(min(N, max_jumps_per_step)):
#                jump_Z = quasi_random_numbers[idx, 1 + j]  # Subsequent dimensions for jumps
#                Y = np.exp(m + np.sqrt(v) * jump_Z)
#                M += np.log(Y)
#
#            X[i] = X[i-1] + (mu - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z + M
#            S[i] = np.exp(X[i])
#        matrix[k] = S
#    return matrix
#
## Usage example
#paths, I = 1000, 252  # Example values for paths and time steps
#max_jumps_per_step = 5  # Maximum number of jumps to consider per time step
#total_dimensions = 1 + max_jumps_per_step  # Dimensions for GBM + jumps
#quasi_random_numbers = generate_quasi_random_normal(paths, I, total_dimensions)



In [163]:
#ln_paths = merton_jump_paths(S0, paths, LN_lam, LN_sigma, LN_mu, LN_v, m, T, I)
#quasi_ln_paths= merton_jump_paths_sobol(S0, paths, LN_lam, LN_sigma, LN_mu, LN_v, m, T, I, max_jumps_per_step, quasi_random_numbers)

In [164]:
#figsize = (10, 6)
#fig = plt.figure(figsize=figsize)
#plt.plot(ln_paths.T[:, :10], lw=1.5)
#plt.xlabel('time')


In [165]:

#V0_sobol, V0_array_sobol, SE_sobol, variance_sobol = LSM(K, quasi_ln_paths, I, np.exp(-r * T), basis, 3)
#V0, V0_array, SE, variance = LSM(K, ln_paths, I, np.exp(-r * T), basis, 3)
#
#print('V0: ', V0)
#print('SE: ', SE)
#print('Variance: ', variance)
#
#print('V0_sobol: ', V0_sobol)
#print('SE_sobol: ', SE_sobol)
#print('Variance_sobol: ', variance_sobol)
